# Pipeline for Recovering Neural Activity from Calcium Imaging Data

### Relevant Citations / Dependencies
**Motion Correction ->** Suite2P Package from Stringer Lab <br>
**ROI Identification ->** Cellpose Package from Stringer Lab <br>
**Signal Extraction ->** FISSA Package from Rochefort Lab <br>
**Source-Separation ->** FISSA Package from Rochefort Lab <br>
**Spike Inference ->** Cascade Package from Helmchen Lab <br>

### Import Modules

In [ ]:
# FISSA: Signal Extraction & Source-Separation
from AnalysisModules.FissaAnalysis import FissaModule
# Assorted Static Processing Functions
from AnalysisModules.StaticProcessing import Processing

### Instantiation

In [ ]:
# Instantiate Fissa Module & Sub-Modules. 
# Sub-Module 1 is preparation, will contain raw data (PreparationModule)
# Sub-Module 2 is experiment, will contain separation data (SeparationModule)
# Sub-Module 3 is ProcessedTraces, just a container for processed signals
Fissa = FissaModule(data_folder="D:\\M4618_Second_Round\\Encoding", index_file = "D:\\M4618_Second_Round\\Encoding\\suite2p\\plane0\\Debug_Neuronal_Index.csv")
# This folder contains the suite2p/plane0/___.npy files as well as the saved registered files located in the suite2p/plane0/reg_tif folder
# Initialize
Fissa.pruneNonNeuronalROIs() # This step removes all non-neuronal data
Fissa.initializeFissa()


### Extract Traces

In [ ]:
Fissa.extractTraces() # simple, call to extract raw traces from videos

### Save Progress

In [ ]:
Fissa.saveFissaPrep()

### Pre-Process Traces

In [ ]:
# let's smooth the data to make it nicer
Fissa.ProcessedTraces.smoothed_raw = Processing.smoothTraces_TiffOrg(Fissa.preparation.raw, niter=50, kappa=150, gamma=0.15)[0]

### Save Progress

In [ ]:
Fissa.saveProcessedTraces()

### Use Pre-Processed Traces for Source-Separation

In [ ]:
Fissa.preparation.raw = Fissa.ProcessedTraces.smoothed_raw.copy()
#Let's use for separation, so replace the raws with smooths
Fissa.passPrepToFissa()

### Source-Separate Traces

In [ ]:
Fissa.separateTraces() # simple, call to separate the traces

### Save Progress

In [ ]:
Fissa.saveFissaSep()

### Post-Process Traces

In [ ]:
# Calculate Fo/F
Fissa.ProcessedTraces.dFoF_result = Processing.calculate_dFoF(Fissa.experiment.result, Fissa.frame_rate, raw=Fissa.preparation.raw, merge_after=False)
# Condense the ROI Traces for each Trial into a Single Matrix
Fissa.ProcessedTraces.merged_dFoF_result = Processing.mergeTraces(Fissa.ProcessedTraces.dFoF_result)
# Detrend the Traces by fitting a 4th-order polynomial and subsequently subtracting
Fissa.ProcessedTraces.detrended_merged_dFoF_result = Processing.detrendTraces(Fissa.ProcessedTraces.merged_dFoF_result, order=4, plot=False)

### Save Progress

In [ ]:
Fissa.saveProcessedTraces()

## Infer Firing Rates & Approximate Spikes

### Import Modules

In [ ]:
# CASCADE: Spike Inference
from AnalysisModules.CascadeAnalysis import CascadeModule

# Assorted Static Utility Functions
from AnalysisModules.StaticUtilities import pullModels

If returning, use this cell to load prior data

In [ ]:
# Instantiate
#Data = FissaModule(data_folder="H:\\DEM_Excitatory_Study\\DEM2\\Retrieval\\Imaging\\10Hz", index_file="H:\\DEM_Excitatory_Study\\DEM2\\Retrieval\\Imaging\\10Hz\\NeuronalIndex.csv")
# Initialize
#Data.initializeFissa()
# Load
#Data.loadFissaPrep()
#Data.loadFissaSep()
#Data.loadProcessedTraces()

### Instantiation

In [ ]:
# Pass the Post-Processed Traces. Here we choose the detrended, matrix-form, Fo/F traces
Cascade = CascadeModule(Fissa.ProcessedTraces.detrended_merged_dFoF_result, Fissa.frame_rate, model_folder="C:\\ProgramData\\Anaconda3\\envs\\suite2p\\Pretrained_models")

### Model Selection

In [ ]:
# Pull Available Models
list_of_models = pullModels(Cascade.model_folder)

# Select Model: If you know what model you want, you should use the string instead.
# This model is GLobal_EXC_10Hz_smoothing_100ms
Cascade.model_name = list_of_models[21]

# Download Model: You will need to download the model before first use
# from AnalysisModules.StaticUtilities import downloadModel

### Infer Firing Rates

In [ ]:
# Infer Spike Probability
Cascade.predictSpikeProb() # Simple, call to infer spike probability for each frame
# Calculate Firing Rates # Simple, firing rate = spike probability * imaging frequency
Cascade.ProcessedInferences.firing_rates = Processing.calculateFiringRate(Cascade.spike_prob, Cascade.frame_rate)

### Save Progress

In [ ]:
Cascade.saveSpikeProb(Fissa.output_folder)
Cascade.saveProcessedInferences(Fissa.output_folder)

### Infer Discrete Spike Times

In [ ]:

# Infer the discrete spike events, simply call
Cascade.inferDiscreteSpikes()

# If you have an infinite loop:
# you are looking at a neuron that should have been dropped for
# (1) exceedingly poor SNR
# (2) major trend in the data (e.g., lots of bleaching)
# (3) abnormal phenotype for particular model-- think of training your own!

### Save Progress

In [ ]:
Cascade.saveSpikeInference(Fissa.output_folder)

### Export to MATLab for MATLab Analyses

In [ ]:
# Here, we can export the spike prob or events to matlab .mat files
Cascade.exportSpikeProb(Fissa.output_folder)
Cascade.exportSpikeInference(Fissa.output_folder)

# Visualization

### Import

In [ ]:
import matplotlib
%matplotlib inline

# Assorted Static Plotting Functions are Found in Static Plotting Module

### Plot Histogram of Noise

In [ ]:
from AnalysisModules.StaticPlotting import plotNoise
plotNoise(Cascade.traces, Cascade.frame_rate)

### Visualize Spike Inference

In [ ]:
from AnalysisModules.StaticPlotting import plotSpikeInference
plotSpikeInference(Cascade.spike_prob, Cascade.spike_time_estimates, Cascade.traces, Cascade.frame_rate)

### Plot Inferred Firing-Rate Matrix

In [ ]:
from AnalysisModules.StaticPlotting import plotFiringRateMatrix
plotFiringRateMatrix(Cascade.ProcessedInferences.firing_rates, Cascade.frame_rate)

### Assess Inference of Spike Probability & Approximation of Discrete Spikes

In [ ]:
from AnalysisModules.StaticPlotting import assessSpikeInference
assessSpikeInference(Cascade.spike_prob, Cascade.spike_time_estimates, Cascade.traces, Cascade.frame_rate)